In [1]:
import pandas as pd                    
import xml.etree.ElementTree as ET
import unicodedata

N_CVs = 1
linhas = [] # uma linha para cada artigo lido
base_path = '../xml/'

researcher_df = pd.DataFrame(columns=['name', 'research_area', 'program'])
base_research_field = pd.DataFrame(columns=['name', 'research_area'])
institution = pd.DataFrame(columns=['institution_name'])
production = pd.DataFrame(columns=['event_name', 'event_qualis', 'event_type'])
time = pd.DataFrame(columns=['year'])

In [2]:
def normalize_str(source, encoding):
    if encoding  == 'ISO-8859-1':
        target = str.encode(source, 'ISO-8859-1')
        target = target.decode('UTF-8')
    else:
        target = str(source)
        
    target = target.strip()
    
    target = ''.join(c for c in unicodedata.normalize('NFD', target)
                  if unicodedata.category(c) != 'Mn')
    
    return target.upper()

In [3]:
def parse_researcher_data(xmlelement, field_df, debug=False):
    name = normalize_str(xmlelement.get('NOME-COMPLETO'), 'ISO-8859-1')
    program = "PPGC" # Can change later to add a properly method to fetch
    
    field = field_df[field_df['name'] == name].iloc[0]['research_area']
    
    if debug:
        print("RESEARCHER [" + name + "]\t[" + program + "]\t[" + field + "]")
    return [name, field, program]

In [4]:
def parse_production(linhas, elem):
    trabalhos = elem.findall('./TRABALHOS-EM-EVENTOS/TRABALHO-EM-EVENTOS')
    for trabalho in trabalhos:
        dados_basicos = trabalho.find('DADOS-BASICOS-DO-TRABALHO').attrib
        detalhes = trabalho.find('DETALHAMENTO-DO-TRABALHO').attrib
        autores = trabalho.findall('AUTORES')
        if dados_basicos.get('NATUREZA') == 'COMPLETO':
            dados = dict(dados_basicos)
            dados.update(detalhes)
            lst = []
            lst_autores= {}
            for autor in autores:                        
                lst.append(autor.get('NOME-COMPLETO-DO-AUTOR'))                      
            lst_autores['AUTORES'] = lst
            dados.update(lst_autores)                    
            linhas.append(dados)

In [5]:
with open(base_path + "Linhas_de_Pesquisa.csv", "r") as file:
    next(file)
    iter = 0
    for line in file:
        arr = line.split(';')
        base_research_field.loc[iter] = [normalize_str(arr[2], 'UTF-8'), arr[1]]
        iter += 1

In [6]:
for i in range(1,N_CVs+1):
    arquivo = base_path + 'curriculo(' + str(i) + ').xml'
    print("Lendo arquivo: ", arquivo)
    doc_tree_root = ET.parse(arquivo).getroot()
    for elem in doc_tree_root:                
        if elem.tag == 'PRODUCAO-BIBLIOGRAFICA':
            parse_production(linhas, elem)
        # Geração de tabela 
        if elem.tag == 'DADOS-GERAIS':
            researcher_df.loc[i-1] = parse_researcher_data(elem, base_research_field)

Lendo arquivo:  ../xml/curriculo(1).xml
Lendo arquivo:  ../xml/curriculo(2).xml
Lendo arquivo:  ../xml/curriculo(3).xml
Lendo arquivo:  ../xml/curriculo(4).xml
Lendo arquivo:  ../xml/curriculo(5).xml
Lendo arquivo:  ../xml/curriculo(6).xml
Lendo arquivo:  ../xml/curriculo(7).xml
Lendo arquivo:  ../xml/curriculo(8).xml
Lendo arquivo:  ../xml/curriculo(9).xml
Lendo arquivo:  ../xml/curriculo(10).xml
Lendo arquivo:  ../xml/curriculo(11).xml
Lendo arquivo:  ../xml/curriculo(12).xml
Lendo arquivo:  ../xml/curriculo(13).xml
Lendo arquivo:  ../xml/curriculo(14).xml
Lendo arquivo:  ../xml/curriculo(15).xml
Lendo arquivo:  ../xml/curriculo(16).xml
Lendo arquivo:  ../xml/curriculo(17).xml
Lendo arquivo:  ../xml/curriculo(18).xml
Lendo arquivo:  ../xml/curriculo(19).xml
Lendo arquivo:  ../xml/curriculo(20).xml
Lendo arquivo:  ../xml/curriculo(21).xml
Lendo arquivo:  ../xml/curriculo(22).xml
Lendo arquivo:  ../xml/curriculo(23).xml
Lendo arquivo:  ../xml/curriculo(24).xml
Lendo arquivo:  ../xml/cu

In [10]:
## Comita as mudanças para tabelas do banco.
import mysql.connector

cnx = mysql.connector.connect(user='root', password='root', database='EBD10')
cursor = cnx.cursor()

researcher_insert = "INSERT INTO Researcher(researcher_name) VALUES (%s)"
field_insert = """INSERT INTO ResearchField (researchfield_name)
                    SELECT * FROM (SELECT %s) AS tmp
                    WHERE NOT EXISTS (
                        SELECT researchfield_name FROM ResearchField WHERE researchfield_name = %s
                    ) LIMIT 1;"""
program_insert = """INSERT INTO ResearchProgram (program_name)
                    SELECT * FROM (SELECT %s) AS tmp
                    WHERE NOT EXISTS (
                        SELECT program_name FROM ResearchProgram WHERE program_name = %s
                    ) LIMIT 1;"""

for idx, row in researcher_df.iterrows():
    data = (row['name'],)
    cursor.execute(researcher_insert, data)
    data = (row['research_area'], row['research_area'])
    cursor.execute(field_insert, data)
    data = (row['program'],row['program'])
    cursor.execute(program_insert, data)
    
cnx.commit()
cursor.close()
cnx.close()

In [ ]:
import mysql.connector

cnx = mysql.connector.connect(user='root', password='root', database='EBD10')
cursor = cnx.cursor()

cursor.execute("insert into Researcher(researcher_id, researcher_name) values (1, 'Zé')")
cnx.commit()

cursor.close()
cnx.close()

In [8]:
researcher_df

,name,research_area,program
0,RENATO PEREZ RIBAS,Concepção de Circuitos e Sistemas Integrados,PPGC
1,VALTER ROESLER,"Arquiteturas, Protocolos e Gerência de Redes e...",PPGC
2,LUIS DA CUNHA LAMB,Lógica e Modelos de Computação,PPGC
3,INGRID OLIVEIRA DE NUNES,Engenharia de Software,PPGC
4,ERIKA FERNANDES COTA,Engenharia de Software,PPGC
5,CLAUDIO ROSITO JUNG,"Processamento de Imagens, Visão Computacional ...",PPGC
6,KARIN BECKER,Engenharia de Software,PPGC
7,DALTRO JOSE NUNES,Engenharia de Software,PPGC
8,JACOB SCHARCANSKI,"Processamento de Imagens, Visão Computacional ...",PPGC
9,NICOLAS MAILLARD,Computação de Alto Desempenho e Sistemas Distr...,PPGC
